## Imports

In [1]:
import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hsql as hsql
import im_v2.ccxt.data.client.ccxt_clients as imvcdccccl
import im_v2.im_lib_tasks as imvimlita
import im_v2.talos.data.client.talos_clients as imvtdctacl
import im_v2.talos.data.extract.exchange_class as imvtdeexcl

## Functions

In [2]:
def get_data_from_talos_db(start_time, end_time):
    # Set start and end dates.
    start_timestamp = pd.Timestamp(start_time)
    end_timestamp = pd.Timestamp(end_time)

    # Load the data.
    df = talos_extract.download_ohlcv_data(
        currency_pair="BTC-USD",
        exchange="binance",
        start_timestamp=start_timestamp,
        end_timestamp=end_timestamp,
        bar_per_iteration=100,
    )
    df["timestamp"] = df["timestamp"].apply(
        lambda x: hdateti.convert_unix_epoch_to_timestamp(x)
    )
    return df


def get_data_from_ccxt_client(start_time, end_time):
    # Specify the params.
    full_symbol_binance = "binance::BTC_USDT"
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    df = ccxt_client._read_data_for_one_symbol(
        full_symbol_binance, start_time, end_time
    )
    return df


def get_data_from_talos_client(start_time, end_time):
    full_symbol_binance = ["binance::BTC_USDT"]
    df = talos_client.read_data(
        full_symbol_binance,
        start_ts=pd.Timestamp(start_time),
        end_ts=pd.Timestamp(end_time),
    )
    return df

# Talos DB

In [3]:
# Initialize extractor.
talos_extract = imvtdeexcl.TalosExchange("sandbox")

In [4]:
data_talos_db = get_data_from_talos_db(
    "2022-01-01T10:00:24.000000Z", "2022-01-01T10:08:00.000000Z"
)
display(data_talos_db.head(3))
display(data_talos_db.tail(3))

,timestamp,open,high,low,close,volume,ticks,end_download_timestamp
0,2022-01-01 10:01:00+00:00,0,0,0,0,0,0,2022-04-11 07:13:41.943377+00:00
1,2022-01-01 10:02:00+00:00,0,0,0,0,0,0,2022-04-11 07:13:41.943377+00:00
2,2022-01-01 10:03:00+00:00,0,0,0,0,0,0,2022-04-11 07:13:41.943377+00:00


,timestamp,open,high,low,close,volume,ticks,end_download_timestamp
5,2022-01-01 10:06:00+00:00,0,0,0,0,0,0,2022-04-11 07:13:41.943377+00:00
6,2022-01-01 10:07:00+00:00,0,0,0,0,0,0,2022-04-11 07:13:41.943377+00:00
7,2022-01-01 10:08:00+00:00,0,0,0,0,0,0,2022-04-11 07:13:41.943377+00:00


### Talos query summary

Beginning
- If proposing query for __a complete minute__ (e.g., __10:00:00__) - it starts with __exactly mentioned timestamp__ (i.e., __10:00:00__).
- If proposing query for __an incomplete minute__ (e.g., __10:00:36 or 10:00:24__) - it starts with __mentioned timestamp + 1min__ (i.e., __10:01:00__).
   - Since the ohlcv output is blank (equal to zero), it's hard to understand whether volume or prices data changes during incomplete minute query.

End
- If proposing query for __a complete minute__ (e.g., __10:07:00__) - it starts with __exactly mentioned timestamp - 1min__ (i.e., __10:06:00__).
- If proposing query for __an incomplete minute__ (e.g., __10:07:36 or 10:07:24__) - it starts with __exactly mentioned timestamp__ (i.e., __10:07:00__).
- If proposing query for __previous minute + 1min__ (e.g., __10:08:00__) - it starts with __exactly mentioned timestamp - 1min__ (i.e., __10:07:00__).

# Current CCXT client

In [5]:
# Specify the params.
vendor = "CCXT"
universe_version = "v3"
root_dir = "s3://alphamatic-data/data"
extension = "csv.gz"
aws_profile_ccxt = "am"
# Initialize CCXT client.
ccxt_client = imvcdccccl.CcxtCddCsvParquetByAssetClient(
    vendor,
    universe_version,
    True,
    root_dir,
    extension,
    aws_profile=aws_profile_ccxt,
)

In [6]:
data_ccxt_client = get_data_from_ccxt_client(
    "2020-01-01 10:00:02", "2020-01-01 10:08:00"
)

INFO  Reading data for vendor=`CCXT`, exchange id='binance', currencies='BTC_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BTC_USDT.csv.gz'...


In [7]:
display(data_ccxt_client.head(3))
display(data_ccxt_client.tail(3))

,open,high,low,close,volume
timestamp,,,,,
2020-01-01 10:01:00+00:00,7187.12,7190.61,7185.93,7188.86,14.830744
2020-01-01 10:02:00+00:00,7190.41,7191.88,7188.10,7191.47,10.590978
2020-01-01 10:03:00+00:00,7191.92,7195.66,7191.59,7194.09,9.576390


,open,high,low,close,volume
timestamp,,,,,
2020-01-01 10:06:00+00:00,7194.00,7194.99,7193.63,7194.98,8.034727
2020-01-01 10:07:00+00:00,7194.99,7196.27,7194.44,7196.17,15.916280
2020-01-01 10:08:00+00:00,7196.20,7198.08,7195.63,7197.45,7.849214


### Current CCXT client summary

Beginning
- If proposing query for __a complete minute__ (e.g., __10:00:00__) - it starts with __exactly mentioned timestamp__ (i.e., __10:00:00+00:00__).
- If proposing query for __an incomplete minute__ (e.g., __10:00:36 or 10:00:24__) - it starts with __mentioned timestamp + 1min__ (i.e., __10:01:00__).
   - - Since the ohlcv output is available, one can check through volume or prices data that changing the query within a minute (e.g., 10:00:02 or 10:00:45) doesn't affect the numbers, so it means that the timestamp indicates the end of time period.

End
- If proposing query for __a complete minute__ (e.g., __10:07:00__) - it starts with __exactly mentioned timestamp__ (i.e., 10:07:00).
- If proposing query for __an incomplete minute__ (e.g., __10:07:36 or 10:07:24__) - it starts with __exactly mentioned timestamp__ (i.e., __10:07:00__).
- If proposing query for __previous minute + 1min__ (e.g., __10:08:00__) - it starts with __exactly mentioned timestamp__ (i.e., __10:08:00__).

# Current implemented Talos client

In [8]:
env_file = imvimlita.get_db_env_path("dev")
universe_version = "v1"
connection_params = hsql.get_connection_info_from_env_file(env_file)
connection = hsql.get_connection(*connection_params)
table_name = "talos_ohlcv"
talos_client = imvtdctacl.RealTimeSqlTalosClient(
    universe_version, True, connection, table_name
)

In [9]:
df = get_data_from_talos_client(
    "2022-03-16 22:47:50+0000", "2022-03-16 22:54:00+0000"
)
display(df.head(3))
display(df.tail(3))

,open,high,low,close,volume,full_symbol
timestamp,,,,,,
2022-03-16 22:48:00+00:00,40927.09,40953.20,40927.08,40945.00,21.30496,binance::BTC_USDT
2022-03-16 22:49:00+00:00,40945.00,40945.01,40863.09,40873.47,47.10019,binance::BTC_USDT
2022-03-16 22:50:00+00:00,40873.46,40900.14,40873.46,40889.01,26.09286,binance::BTC_USDT


,open,high,low,close,volume,full_symbol
timestamp,,,,,,
2022-03-16 22:52:00+00:00,40922.01,40938.95,40922.00,40930.42,5.54002,binance::BTC_USDT
2022-03-16 22:53:00+00:00,40930.00,40930.14,40889.02,40903.37,13.82792,binance::BTC_USDT
2022-03-16 22:54:00+00:00,40904.00,40906.99,40877.15,40893.66,12.28861,binance::BTC_USDT


### Talos client summary

Beginning
- If proposing query for __a complete minute__ (e.g., __10:00:00__) - it starts with __exactly mentioned timestamp__ (i.e., __10:00:00__).
- If proposing query for __an incomplete minute__ (e.g., __10:00:36 or 10:00:24__) - it starts with __mentioned timestamp + 1min__ (i.e., __10:01:00__).
   - Since the ohlcv output is available, one can check through volume or prices data that changing the query within a minute (e.g., 10:00:02 or 10:00:45) doesn't affect the numbers, so it means that the timestamp indicates end of time period.

End
- If proposing query for __a complete minute__ (e.g., __10:07:00__) - it starts with __exactly mentioned timestamp__ (i.e., __10:07:00__).
- If proposing query for __an incomplete minute__ (e.g., __10:07:36 or 10:07:24__) - it starts with __exactly mentioned timestamp__ (i.e., __10:07:00__).
- If proposing query for __previous minute +1min__ (e.g., __10:08:00__) - it starts with __exactly mentioned timestamp__ (i.e., __10:08:00__).